## EDA (Exploratory Data Analysis) for beginner

It doesn't look like there's much to analyze, so let's take a quick look at the CSV data, see the distribution of the target values, and then output the actual image.

분석할 요소가 많지 않은 것 같습니다. csv 데이터를 간단히 둘러보고, 타깃값 분포를 알아본 뒤 실제 이미지를 출력해보겠습니다.

### 1. Load the data (데이터 불러오기)

For a quick tour of the CSV data, we'll start by importing the CSV file.

csv 데이터를 간단히 둘러보기 위해 우선 csv 파일을 불러옵니다.

In [ ]:
import pandas as pd

data_path = '/kaggle/input/plant-pathology-2020-fgvc7/'

train = pd.read_csv(data_path + 'train.csv')
test = pd.read_csv(data_path + 'test.csv')
submission = pd.read_csv(data_path + 'sample_submission.csv')

train.shape, test.shape

Both the training and test data have the same number of columns, 1,821. The training data has four more columns than the test data because it has four target values.

Next, let's output the first five rows of the training data.

훈련 데이터와 테스트 데이터 모두 1,821개로 개수가 같네요. 타깃값이 4개라서 훈련 데이터의 열 개수가 테스트 데이터보다 4개 더 많습니다.

다음으로 훈련 데이터의 첫 다섯행을 출력해봅시다.

In [ ]:
train.head()

image_id represents the filename of the training image data. This means that the image files for training are named Train_0.jpg, Train_1.jpg, Train_2.jpg, and so on (the image filenames are located in the images directory).

The healthy, multiple_diseases, rust, and scap columns are the target values. They are written in a one-hot encoding format.

image_id는 훈련 이미지 데이터의 파일명을 나타냅니다. 훈련용 이미지 파일의 이름이 Train_0.jpg, Train_1.jpg, Train_2.jpg 식이라는 뜻입니다(이미지 파일명은 images 디렉터리에 있습니다).

healthy, multiple_diseases, rust, scap 열은 타깃값입니다. 원-핫 인코딩 형식으로 기록되어 있습니다.

In [ ]:
test.head()

There are no target values in the test data, so there are only image_ids. There are also 1,821 test data, so the image data filenames would be Test_0.jpg through Test_1820.jpg.

테스트 데이터에는 타깃값이 없으니 image_id만 있습니다. 테스트 데이터도 총 1,821개이므로 이미지 데이터 파일명은 Test_0.jpg부터 Test_1820.jpg까지 있겠네요.

In [ ]:
submission.head()

타깃값 4개의 값이 모두 0.25입니다. 각 확률을 25%로 일괄 기재해둔 것이죠

## 2. Data Visualization (데이터 시각화)

We're going to visualize the data in the CSV file we just looked at.

방금 살펴본 csv 파일의 데이터를 시각화해볼 겁니다.

### [1] Distribution of target value (타깃값 분포)

The first thing we'll do is divide the data by target value to see how many data points correspond to each target value.

We've assigned each of the variables 'healthy', 'multiple_diseases', 'rust', and 'scab' with data from the target value. Let's plot a pie graph of the target value distribution using these variables.

가장 먼저 데이터를 타깃값별로 나눠보겠습니다. 각 타깃값에 해당하는 데이터가 몇 개씩인지 알아보기 위해서입니다.

'healthy', 'multiple_diseases', 'rust', 'scab' 각 변수에 타깃값의 데이터를 할당했습니다. 이 변수들을 사용해 타깃값 분포를 파이 그래프로 그려보겠습니다.

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

mpl.rc('font', size = 15)
plt.figure(figsize = (7, 7))

healthy = train.loc[train['healthy'] == 1]
multiple_diseases = train.loc[train['multiple_diseases'] == 1]
rust = train.loc[train['rust'] == 1]
scab = train.loc[train['scab'] == 1]

label = ['healthy', 'multiple_diseases', 'rust', 'scab']

plt.pie([len(healthy), len(multiple_diseases), len(rust), len(scab)],
         labels = label, autopct = '%.1f%%')

The ratio is rust > scab > healthy > multiple_diseases. There are relatively few multiple_diseases compared to the rest of the targets. Because of the large difference in proportions, it would be better to divide the training and validation data according to the proportions of the target values.

비율이 rust > scab > healthy > multiple_diseases 순입니다. 나머지 타깃값에 비해 multiple_disease가 상대적으로 적습니다. 비율 차이가 크기 때문에 훈련 데이터와 검증 데이터로 나눌 때 타깃값 비율에 맞게 나누는 게 좋겠네요.

### [2] Image Output (이미지 출력)

To display an image for each target value, we'll first define a show_image() function that takes an image ID and displays the image on the screen, find the image ID for each target value to pass to the function, and write code to get the image_id of the last 6 image data for each target value.

Once we have the image IDs, let's use the show_image() function to output the leaf image.

각 타깃값에 해당하는 이미지를 출력해보겠습니다. 이미지 ID를 전달받아 화면에 이미지를 출력하는 show_image() 함수를 먼저 정의하고 이 함수에 전달할 타깃값별 이미지 ID를 구해보겠습니다. 그리고 타깃값별로 마지막 6개 이미지 데이터의 image_id를 구하는 코드를 작성ㅇ합니다.

이미지 ID를 알아내면, 이제 show_image() 함수로 잎사귀 이미지를 출력해봅시다.

In [ ]:
import matplotlib.gridspec as gridspec
import cv2

def show_image(img_ids, rows = 2, cols = 3):
    assert len(img_ids) <= rows * cols # 이미지가 행/열 개수보다 많으면 오류 발생
    
    plt.figure(figsize = (15, 8))
    grid = gridspec.GridSpec(rows, cols)
    
    for idx, img_id in enumerate(img_ids):
        img_path = f'{data_path}/images/{img_id}.jpg'
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        ax = plt.subplot(grid[idx])
        ax.imshow(image)

num_of_imgs = 6
last_healthy_img_ids = healthy['image_id'][-num_of_imgs:]
last_multiple_diseases_img_ids = multiple_diseases['image_id'][-num_of_imgs:]
last_rust_img_ids = rust['image_id'][-num_of_imgs:]
last_scab_img_ids = scab['image_id'][-num_of_imgs:]

show_image(last_healthy_img_ids)

This is a healthy leaf, and it's overall clean.

Next, let's print out a leaf with multiple diseases, a leaf with rust disease, and a leaf with red mold disease.

건강한 잎사귀답게 전반적으로 깨끗하네요.

다음으로 여러 질병에 걸린 잎사귀, 녹병에 걸린 잎사귀와 붉은곰팡이병에 걸린 잎사귀를 출력해봅시다.

In [ ]:
show_image(last_multiple_diseases_img_ids)
show_image(last_rust_img_ids)
show_image(last_scab_img_ids)

## 3. Analysis Summary & Modeling Strategy

### Analysis Summary (분석 정리)

1. The identity feature of the CSV file is the image filename. We just need to add the pathname and extension of the file to get the location of the file.
2. In the training data, we recorded the leaf states (target values) in the form of one-hot encoding divided into four columns.
3. Since the ratio difference between the target values is large, we need to divide the training data and the validation data according to the ratio of the target values.

1. csv 파일의 id 피처는 이미지 파일명입니다. 파일의 경로명과 확장명만 추가하면 파일의 위치를 바로 얻어올 수 있습니다.
2. 훈련 데이터에는 잎사귀 상태(타깃값)를 4개 열로 나눠 원-핫 인코딩 형태로 기록해뒀습니다.
3. 타깃값들의 비율 차이가 커서 훈련 데이터와 검증 데이터를 나눌 때 타깃값 비율에 맞게 나눠야 합니다.

### Modeling Strategy (모델링 전략)

Instead of designing our own deep learning model, we'll use a pre-trained model known to perform well to perform transfer learning. We'll focus on learning useful performance improvement techniques outside of neural network design.

- Baseline model
  - Data augmentation: applying various transformers
  - Neural Network Structure: Pre-trained model (efficientnet-b7)
  - Optimizer: AdamW

- Performance Improvements
  - Data augmentation: Same as baseline
  - Neural network structure: same as baseline
  - Optimizer: Same as baseline
  - Training phase optimizations: Scheduler settings, increased epochs
  - Prediction phase optimizations: Test phase data augmentation (TTA), label smoothing

딥러닝 모델을 직접 설계하지 않고, 성능이 우수하다고 알려진 사전 훈련 모델을 활용해 전이 학습을 수행하겠습니다. 신경망 설계 외의 유용한 성능 향상 기법들을 배우는 데 집중하려고 합니다.

- 베이스라인 모델
  - 데이터 증강 : 다양한 변환기 적용
  - 신경망 구조 : 사전 훈련 모델(efficientnet-b7)
  - 옵티마이저 : AdamW


- 성능 개선
  - 데이터 증강 : 베이스라인과 동일
  - 신경망 구조 : 베이스라인과 동일
  - 옵티마이저 : 베이스라인과 동일
  - 훈련 단계 최적화 : 스케줄러 설정, 에폭 증가
  - 예측 단계 최적화 : 테스트 단계 데이터 증강(TTA), 레이블 스무딩

The End.